# Ejemplo: Búsqueda Tabú (BT)

## Problema de la mochila binario

El problema de la mochila binario (0/1 Knapsack problem) considera un conjunto finito de $n$ objetos, donde cada objeto $i$ tiene un valor $p_i$ y un peso $w_i$, y una mochila que puede soportar hasta un peso determinado $c$. El objetivo es encontrar el subconjunto de objetos que puedan ser transportados en la mochila, maximizando el valor de la mochila. Formalmente se define como sigue:

\begin{equation}
  \label{eq:KP}
  \begin{array}{rll}
  \text{maximizar:} & f(\vec{x}) = \sum_{i=1}^{n} p_i \cdot x_{i} &  \\
  \text{tal que} & g_1(\vec{x}) = \sum_{i=1}^{n} w_i \cdot x_{i}  \leq c &  \\
          &  x_i \in \{0,1\} & i\in\{1,\ldots,n\}\\
  \end{array}
\end{equation}

Para este ejemplo, consideremos que se tienen $n=5$ objetos. Donde cada objeto tiene los siguientes valores $p=[5,14,7,2,23]$ y los siguientes pesos $w = [ 2, 3, 7, 5, 10 ]$. Además la mochila tiene una capacidad $c = 15$.

## Principales componentes de BT

Librerías de Python que vamos a utilizar.

In [1]:
import numpy
import math

Datos de entrada.

In [2]:
n=5
p = [5,14,7,2,23]
w = [2,3,7,5,10]
c = 15

### Representación de una solución

Cada solución es una cadena binaria de tamaño 5. Para facilitar la implementación en Python, lo tomaremos como una lista de 0's y 1's. Por ejemplo, la solución x='11001' es almacenada como:

In [3]:
x = [1,1,0,0,1]

### Solución inicial

Mientras no se exceda la capacidad de la mochila, se van introduciendo de manera aleatoria objetos a la mochila.

In [4]:
def getInitialSolution(n, p, w, c):
    #Ningún objeto está en la mochila
    x = [0 for i in range(n)]
    weight_x = 0
    
    #Aleatoriamente elegimos el orden en el que intentaremos introducir 
    #los objetos a la mochila
    objects = list(range(n))
    numpy.random.shuffle(objects)
    
    for o in objects:
        #Intentamos introducir el objeto "o" a la mochila
        if weight_x + w[o] <= c:
            x[o] = 1
            weight_x += w[o]
            
    return x

In [5]:
x_0 = getInitialSolution(n, p, w, c)
x_0

[0, 0, 0, 1, 1]

### Función objetivo

In [6]:
def f(p, x):
    profits = 0
    for i in range(len(x)):
        profits += p[i]*x[i]
        
    return profits

In [7]:
print ("f(x): \t", f(p, x))
print ("f(x_0): ", f(p, x_0))

f(x): 	 42
f(x_0):  25


### Restricción

In [8]:
def g1(w, x):
    weight = 0
    for i in range(len(x)):
        weight += w[i]*x[i]
        
    return weight

In [9]:
print ("g1(w, x): \t", g1(w, x))
print ("g1(w, x_o): \t", g1(w, x_0))

g1(w, x): 	 15
g1(w, x_o): 	 15


### Movimiento

Altera el valor del elemento ubicado en la posición $i$ de nuestra solución $x$.

In [11]:
def bitflip(x, i):
    x_new = x.copy()
    if x[i] == 0:
        x_new[i] = 1
    else:
        x_new[i] = 0
    
    return x_new

In [12]:
print("x: \t", x)
x_new = bitflip(x, 2)
print("x_new: \t", x_new)

x: 	 [1, 1, 0, 0, 1]
x_new: 	 [1, 1, 1, 0, 1]


### Vecindario

El vecindario está definido como movimientos "bitflip" en cada una de las variables de la solución, teniendo en cuenta que solamente las soluciones factibles pueden ser parte del vecindario. En Python vamos a implementar el vecindario como una lista de listas, donde cada lista interna almacena una solución y su valor tanto de la función objetivo como de la restricción.

In [15]:
def getNeighborhood(p, w, x, c):
    neighborhood = []
    for i in range(len(x)):
        x_new = bitflip(x, i)
        g1_x_new = g1(w, x_new)
        #Si la solución creada es factible, la metemos al vecindario
        if g1_x_new <= c:
            neighborhood.append([x_new, f(p, x_new), g1_x_new, i])
    
    return neighborhood

In [16]:
print("Vecindario de ", x)
getNeighborhood(p, w, x, c)

Vecindario de  [1, 1, 0, 0, 1]


[[[0, 1, 0, 0, 1], 37, 13, 0],
 [[1, 0, 0, 0, 1], 28, 12, 1],
 [[1, 1, 0, 0, 0], 19, 5, 4]]

Es importante mencionar que para instancias grandes del problema (n's grandes) se debe cuidar la eficiencia. Por ejemplo, no es necesario recalcular completamente los valores de la función objetivo y la restricción de cada solución nueva. Se puede obtener a partir de la solución base y el cambio realizado.

### Lista tabú

La lista tabú va a almacenar los valores que no son permitidos en ciertas posiciones de la solución y su "tiempo tabú". En Python utilizaremos un diccionario para guardar los elementos tabú. La llave del diccionario será la posición $p$ y va estar asociada con el valor $v$ que está prohibido en esa posición y con su tiempo tabú $t$. El siguiente ejemplo indica que en la posición $1$ no puede haber un valor de $0$ durante las próximas $2$ iteraciones.

In [17]:
tabu_list = {}
tabu_list[1] = [0, 2]
print (tabu_list)

{1: [0, 2]}


La lista tabú se va a modificar cada iteración actualizando el tiempo tabú de los elementos que ya se encontraban en ella y añadiendo un nuevo elemento.

In [18]:
def updateTabuList(new_element, tabu_list):
    aux = []
    
    #Disminuimos en 1 el tiempo tabú de los elementos que ya estaban en la 
    #lista tabú
    for key in tabu_list:
        tabu_list[key][1] -= 1
        if tabu_list[key][1] == 0:
            aux.append(key)
    
    #Sacamos de la lista tabú los elementos con tiempo tabú igual con 0
    for key in aux:
        tabu_list.pop(key)
        
    #Agregamos el nuevo elemento a la lista tabú    
    tabu_list[new_element[0]] = [new_element[1], new_element[2]]

In [19]:
print(tabu_list)
tabu_element = [3,1,2]
updateTabuList(tabu_element, tabu_list)
print(tabu_list)
tabu_element = [2,0,2]
updateTabuList(tabu_element, tabu_list)
print(tabu_list)

{1: [0, 2]}
{1: [0, 1], 3: [1, 2]}
{3: [1, 1], 2: [0, 2]}


### Vecindario reducido

El vecindario reducido se define como el conjunto de soluciones resultante de quitar del vecindario las soluciones generadas a partir de movimientos que se encuentran en la lista tabú.

In [85]:
def getReducedNeighborhood(x, tabu_list, p, w, c):
    neighborhood = []
    for i in range(len(x)):
        '''Si no está prohibido cambiar el valor de la posición i, 
        creamos una nueva solución haciendo bitflip en la posición i'''
        if i not in tabu_list:
            x_new = bitflip(x, i)
            g1_x_new = g1(w, x_new)
            #Si la solución creada es factible, la metemos al vecindario
            if g1_x_new <= c:
                neighborhood.append([x_new, f(p, x_new), g1_x_new, i])
    
    return neighborhood

## Algoritmo completo de una BT simple

In [86]:
#Obtenemos el índice de la mejor solución en el vecindario
def getIndexBestNeighbor(neighborhood):
    best = 0
    for i in range(1, len(neighborhood)):
        if neighborhood[i][1] >= neighborhood[best][1]:
            best = i
            
    return best

In [87]:
#Búsqueda tabú simple para el problema de la mochila binario
def TabuSearch(num_ite, n, p, w, c):
    #Obtenemos la solución inicial
    x = getInitialSolution(n, p, w, c)
    f_x = f(p, x)
    g_x = g1(w, x)
    
    #Hasta ahora la solución inicial es la mejor solución que se conoce
    x_best, f_best, g_best = x.copy(), f_x, g_x
    
    #Iniciamos con nuestra lista tabú vacía
    tabu_list = {}
    
    #Definimos el tiempo tabú
    tabu_time = n//2
    
    for k in range(num_ite):
        #Obtenemos nuestro vecindario reducido
        neighborhood =  getReducedNeighborhood(x, tabu_list, p, w, c)
        #Si nuestro vecindario está vacío, ya no podemos movernos a otra 
        #solución y debemos terminar la búsqueda
        if len(neighborhood) == 0:
            break
        
        #Nuestra siguiente solución es la mejor del vecindario
        best_neighbor = getIndexBestNeighbor(neighborhood)
        x = neighborhood[best_neighbor][0]
        f_x = neighborhood[best_neighbor][1]
        g_x = neighborhood[best_neighbor][2]
        
        #Verificamos si la nueva solución es mejor que lo que conocemos
        if f_x >= f_best:
            x_best, f_best, g_best = x.copy(), f_x, g_x
        
        #Actualizamos la lista tabú
        i = neighborhood[best_neighbor][3]
        tabu_element = [i, x[i], tabu_time]
        updateTabuList(tabu_element, tabu_list)
        
    return x_best, f_best, g_best
        

In [88]:
def printSolution(x, f, g):
    print("f(x) = ", f, "\tg(x) = ", g)
    step = 25
    print("x = \n[", end="")
    for i in range(len(x)-1):
        if i != 0 and i % step == 0:
            print()
        print(x[i], end=', ')
    print(x[-1], "]")
        

In [89]:
n=5
p = [5,14,7,2,23]
w = [2,3,7,5,10]
c = 15
num_ite = 50

x_best, f_best, g_best = TabuSearch(num_ite, n, p, w, c)
printSolution(x_best, f_best, g_best)


f(x) =  42 	g(x) =  15
x = 
[1, 1, 0, 0, 1 ]


Ejecutamos para una instancia aleatoria más grande del problema.

In [90]:
n=50
profits = list(range(30, 100))
weights = list(range(20, 40))

p = numpy.random.choice(profits, n)
print("Arreglo con valores de cada objeto: \n", p)
w = numpy.random.choice(weights, n)
print("Arreglo con pesos de cada objeto: \n", w)
c = numpy.random.randint((30*n)//2, 30*n)
print("Capacidad de la mochila: ", c)
num_ite = 1000

print("Ejecución 1:")
x_best, f_best, g_best = TabuSearch(num_ite, n, p, w, c)
printSolution(x_best, f_best, g_best)

Arreglo con valores de cada objeto: 
 [47 59 37 63 68 71 99 38 99 38 78 33 89 63 74 92 32 78 31 89 73 92 38 30
 38 30 66 68 45 30 72 73 41 51 88 65 85 76 65 40 67 62 96 68 30 81 67 42
 53 97]
Arreglo con pesos de cada objeto: 
 [38 24 35 34 37 35 29 24 35 34 32 37 34 37 33 34 22 39 36 32 38 26 24 29
 20 32 26 39 29 22 22 33 22 24 21 23 22 32 39 38 38 37 34 27 34 22 29 37
 30 31]
Capacidad de la mochila:  1337
Ejecución 1:
f(x) =  2891 	g(x) =  1329
x = 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 
0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1 ]


Realizamos $m$ ejecuciones de nuestro algoritmo y obtenemos la corrida que encontró la mejor solución, la corrida que encontró la peor solución, la media y la desviación estándar de las $m$ corridas con respecto al valor de la función objetivo.

In [94]:
m=21
sol = []
for i in range(m):
    print("Ejecución ", i, ": ")
    x_best, f_best, g_best = TabuSearch(num_ite, n, p, w, c)
    sol.append([f_best, g_best, x_best])
    printSolution(x_best, f_best, g_best)
    
sol.sort()
print("*************** Mejor solución ***************")
printSolution(sol[-1][2], sol[-1][0], sol[-1][1])

print("*************** Peor solución ****************")
printSolution(sol[0][2], sol[0][0], sol[0][1])

print("****************** Mediana *******************")
med = m//2
printSolution(sol[med][2], sol[med][0], sol[med][1])

f_sol = [x[0] for x in sol]
print("Media: ", numpy.mean(f_sol))
print("Desviación estándar: ", numpy.std(f_sol))

Ejecución  0 : 
f(x) =  2911 	g(x) =  1336
x = 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 
0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1 ]
Ejecución  1 : 
f(x) =  2891 	g(x) =  1329
x = 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 
0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1 ]
Ejecución  2 : 
f(x) =  2891 	g(x) =  1329
x = 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 
0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1 ]
Ejecución  3 : 
f(x) =  2891 	g(x) =  1329
x = 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 
0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1 ]
Ejecución  4 : 
f(x) =  2891 	g(x) =  1329
x = 
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 
0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,